In [1]:
#auch kein sampling,  nur class weight
%store -r handannotated_texts
%store -r handannotated_labels

In [40]:
#test for hashtags
%store -r agreements
nonemotional = []
emotional = []
for t in agreements:
    if t[1] == "0":
        nonemotional.append(t[0])
    elif t[1] == "1":
        emotional.append(t[0])
for post in emotional:
    if "#jhj" in post:
        print(post)
    elif "#geschichteammorgen" in post:
        print(post)
print("#######################################################")        
for post in nonemotional:
    if "#mimimi" in post:
        print(post)
    elif "#DBakel" in post:
        print(post)
    elif "#dbakel" in post:
        print(post)
    elif "#DBakel" in post:
        print(post)
    elif "#dankemerkel" in post:
        print(post)
    elif "#dankefürnix" in post:
        print(post)
    elif "#wudel" in post:
        print(post)
    elif "#Wudel" in post:
        print(post)
    elif "#verzweifelt" in post:
        print(post)
        


#######################################################


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [3]:
#######clean#######

In [4]:
from num2words import num2words
#convert everything to lowercase
handannotated_texts = [x.lower() for x in handannotated_texts]
#exchange numbers with their string representation
#make numbers to text
temp = []
without_num = []
for post in handannotated_texts:
    for word in post.split(" "):
        if word.isdigit():
            #print(word)
            word = num2words(word, lang="de")
        temp.append(word)
    without_num.append((' '.join(temp)))
    temp = []
handannotated_texts = without_num

In [5]:
###############feature extraction########

In [6]:
#emoji feature
emojis = ["😷","😫","🤮", "😣", "😕", "😍", "😞", "😢", "😂", "🥰", "😭",
          "😱", "😒", "😔","😕", "😖", "😫", "😤", "😠", "😡", "🤬","😪"]
from copy import deepcopy
contains_emotional_emojis = deepcopy(handannotated_texts) #FEATURE
#print(contains_emotional_emojis)
for n, post in enumerate(contains_emotional_emojis):
    for e in emojis:
        if e in post:
            #print(post)
            contains_emotional_emojis[n] = [1]
for n,i in enumerate(contains_emotional_emojis):
    if not i == [1]:
        contains_emotional_emojis[n] = [0] 

In [7]:
#hashtag-features

#EMOTIONAL
binary_emotional_hashtags = deepcopy(handannotated_texts) #FEATURE
for i, post in enumerate(binary_emotional_hashtags):
    if "#mimimi" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#DBakel" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#dbakel" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#DBakel" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#dankemerkel" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#dankefürnix" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#wudel" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#Wudel" in post:
        binary_emotional_hashtags[i] = [1]
    elif "#verzweifelt" in post:
        binary_emotional_hashtags[i] = [1]
        
for i, post in enumerate(binary_emotional_hashtags):
    if not post == [1]:
        binary_emotional_hashtags[i] = [0]
        
#NON EMOTIONAL
binary_non_emotional_hashtags = deepcopy(handannotated_texts) #FEATURE
for i, post in enumerate(binary_non_emotional_hashtags):
    if "#jhj" in post:
        binary_non_emotional_hashtags[i] = [1]
    elif "#geschichteammorgen" in post:
        binary_non_emotional_hashtags[i] = [1]
for i, post in enumerate(binary_non_emotional_hashtags):
    if not post == [1]:
        binary_non_emotional_hashtags[i] = [0]        
binary_non_emotional_hashtags        

num_of_hashtags = 0
for li in binary_emotional_hashtags:
    for i in li:
        if i == 1:
            num_of_hashtags +=1

for li in binary_non_emotional_hashtags:
    for i in li:
        if i == 1:
            num_of_hashtags +=1            
            
num_of_hashtags
    

8

In [8]:
#without deleting stopwords
#ngram feature matrices

#unigram
uni_vectorizer = CountVectorizer(ngram_range=(1,1))
unigram_feature_vector = uni_vectorizer.fit_transform(handannotated_texts).toarray() 

#uni and bigram
uni_bi_vectorizer = CountVectorizer(ngram_range=(1,2))
unigram_bi_feature_vector = uni_bi_vectorizer.fit_transform(handannotated_texts).toarray() 

#bigram
bi_vectorizer = CountVectorizer(ngram_range=(2,2))
bigram_feature_vector = bi_vectorizer.fit_transform(handannotated_texts).toarray()


#trigram
tri_vectorizer = CountVectorizer(ngram_range=(3,3))
trigram_feature_vector = tri_vectorizer.fit_transform(handannotated_texts).toarray() 

In [9]:
#tfidf vectorizer ohne stopwords zu löschen
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(handannotated_texts).toarray() #feature matrix

In [10]:
#numerical features
#average length of post, average sentence length, average number of punctuation
#punctuation weg lassen weil nicht wirklich aussagekräftig


import numpy
import string
from nltk import tokenize

post_length = [] #feature vector
counter = 0
number_of_punct = [] #feature vector
sentence_lengths = [] 
tokenized_posts = []
for text in handannotated_texts:
    post_length.append([len(text.split())])
    tokenized_posts.append(tokenize.sent_tokenize(text))
    for char in text:
        if char in string.punctuation:
            counter+=1
    number_of_punct.append([counter])
    counter = 0
sent = []
for post in tokenized_posts:
    for sentence in post:
        sent.append(len(sentence.split()))
    sentence_lengths.append(sent)
    sent = []
#get average
average_sent_len = [] #feature vector
for post in sentence_lengths:
    average_sent_len.append([sum(post)/len(post)])    
print(len(sentence_lengths)) #why 2169?? need:1269 need one number, maybe the average sentence length?
#print(average_sent_len)
#print(sentence_lengths)

97


In [11]:
#POS features
import spacy
nlp = spacy.load('de_core_news_md')

#get POStags via spacy
spacy_obj = []
post_as_pos_tags = []
as_pos = [] #this
for text in handannotated_texts:
    spacy_obj.append(nlp(text))
for text in spacy_obj:
    for token in text:
        post_as_pos_tags.append(token.tag_)
    as_pos.append(' '.join(post_as_pos_tags))
    post_as_pos_tags = []

#neue POS-features
ADV_count = []
PPER_count = []
comma_count = []
NE_count = []
NN_count = []
ADJD_count = []
for post in as_pos:
    ADV_count.append([post.count("ADV")])
    PPER_count.append([post.count("PPER")])
    comma_count.append([post.count("$,")])
    NE_count.append([post.count("NE")])
    NN_count.append([post.count("NN")])
    ADJD_count.append([post.count("ADJD")])
    
print(len(ADJD_count))

#EXPERIMENTS
#get posts as sequence of pos tags
#as_pos = sequence of pos tags
#make sequences into count vectorizers
pos_vec_uni = CountVectorizer(ngram_range=(1,1))
pos_uni_count_vec = pos_vec_uni.fit_transform(as_pos).toarray() #uni feature vector
pos_vec_bi = CountVectorizer(ngram_range=(2,2))
pos_bi_count_vec = pos_vec_bi.fit_transform(as_pos).toarray() #bi feature vector
#make theminto tfidf vectorizers
tfidf_vectorizer_pos = TfidfVectorizer()
tfidf_pos = tfidf_vectorizer_pos.fit_transform(as_pos) #tfidf pos feature matrix hat das iwelche benefits??

97


In [12]:
#document vectors as features

spacy_posts = []
vector_posts = []
temp_post = []
for post in handannotated_texts:
    spacy_posts.append(nlp(post))
for post in spacy_posts:
    for word in post:
        temp_post.append(word.vector)
    vector_posts.append(temp_post)
    temp_post = []
    
mean_vector_posts = []
for post_list in vector_posts:
    #print(len(post))
    mean_vector_posts.append([sum(post_list)/len(post_list)])
    
word_vector_feature = numpy.concatenate(mean_vector_posts, axis=0)
print(word_vector_feature.shape)   


(97, 300)


In [13]:
#wordvectors as features

#make all to numpy arrays
np_posts = []
for p in vector_posts:
    np_posts.append(numpy.array(p))


#get matrix
#test
padded_arrays = []
for p in np_posts:
    padded_arrays.append(numpy.pad(p, ((0, 48 - p.shape[0]), (0, 0)), 'constant', constant_values=0))


print(len(padded_arrays))
np_pad_arr = numpy.array(padded_arrays)
np_pad_arr.shape #shape (97, 48, 300)

nodoc = numpy.reshape (np_pad_arr, (97, -1)) #FEATURE VECTOR FOR WORDVECTORS
print("from ", np_pad_arr.shape, " to ", nodoc.shape)

97
from  (97, 48, 300)  to  (97, 14400)


In [14]:
###################combine feature-vectors:##########################

In [15]:
#combine feature-vectors:

#uni-grams
#unigram_feature_vector

#bigrams
#bigram_feature_vector

#trigrams
#trigram_feature_vector

#unigrams and bigrams
#unigram_bigram_feature_vector

#tfidf

#alle POS-features
pos = numpy.append(ADV_count, comma_count, 1)
pos = numpy.append(pos, NE_count, 1)
pos = numpy.append(pos, NN_count, 1)
pos = numpy.append(pos, ADJD_count, 1)
pos = numpy.append(pos, PPER_count, 1)

#numerical features
#sentence_length
post_sent = numpy.append(post_length, average_sent_len, 1)
post_sent_punct = numpy.append(post_sent, number_of_punct, 1)

#documents vectors
#word_vector_feature

#wordvectors
#nodoc

#emoji feature
#contains_emotional_emojis

#hashtag feature
hash_ = numpy.append(binary_emotional_hashtags, binary_non_emotional_hashtags, 1)


# 3) uni-grams, bigrams, trigrams
uni_bi_tri_feat_vec = numpy.append(unigram_feature_vector, bigram_feature_vector, 1)
uni_bi_tri_feat_vec = numpy.append(uni_bi_tri_feat_vec,trigram_feature_vector, 1)

# 4) tfidf and bigrams
tfidf_bigram = numpy.append(tfidf, bigram_feature_vector, 1)

# 5) unigrams and POS features
uni_pos_new = numpy.append(unigram_feature_vector, pos, 1)

# 6) unigrams and numerical features
uni_num = numpy.append(unigram_feature_vector, post_sent_punct, 1)

# 7) unigrams and POS tags and numerical features
uni_num_pos = numpy.append(uni_num, pos, 1)

# 8) Bigrams and POS
bi_pos_new = numpy.append(bigram_feature_vector, pos, 1)

# 9) bigrams and numerical features
bi_num = numpy.append(bigram_feature_vector, post_sent_punct, 1)

# 10) bigrams and POS tags and numerical features
bi_num_pos = numpy.append(bi_num, pos, 1)

# 11) tfidf and POS
tfidf_pos_new = numpy.append(tfidf, pos, 1)

# 12) tfidf and numerical features
tfidf_num = numpy.append(tfidf, post_sent_punct, 1)

# 13) tfidf and POS tags and numerical features
tfidf_num_pos = numpy.append(tfidf_num, pos, 1)

# 14) wordvec and POS
wordvec_pos = numpy.append(nodoc, pos, 1)

# 15) wordvec and numerical features
wordvec_num = numpy.append(nodoc, post_sent_punct, 1)

# 16) wordvec and POS tags and numerical features
wordvec_num_pos = numpy.append(wordvec_num, pos, 1)

# 17) docvec and POS
docvec_pos = numpy.append(word_vector_feature, pos, 1)

# 18) docvec and numerical features
docvec_num = numpy.append(word_vector_feature, post_sent_punct, 1)

# 19) docvec and POS tags and numerical features
docvec_num_pos = numpy.append(docvec_num, pos, 1)


# 20)POS and numerical features
pos_post_sent_len = numpy.append(pos, post_sent, 1)
pos_num = numpy.append(post_sent_punct, pos, 1)

# 21) alle
all_features = numpy.append(unigram_feature_vector, bigram_feature_vector, 1)
all_features = numpy.append(all_features, trigram_feature_vector , 1)
all_features = numpy.append(all_features, tfidf, 1)
all_features = numpy.append(all_features,post_sent_punct, 1)
all_features = numpy.append(all_features, pos, 1)
all_features = numpy.append(all_features,nodoc, 1)
all_features = numpy.append(all_features,word_vector_feature, 1)
all_features = numpy.append(all_features,contains_emotional_emojis, 1)
all_features = numpy.append(all_features,hash_ ,1)

# 22) hash unigram
emot_uni = numpy.append(unigram_feature_vector, contains_emotional_emojis, 1)
# 23) hash bigram
emot_bi = numpy.append(bigram_feature_vector,contains_emotional_emojis, 1)
# 24) hash tfidf
emot_tfidf = numpy.append(tfidf,contains_emotional_emojis, 1)
# 25) hash wordvectors
emot_wordvec = numpy.append(nodoc,contains_emotional_emojis, 1)
# 26) hash docvecs
emot_docvec = numpy.append(word_vector_feature,contains_emotional_emojis, 1)

# 27) hash unigram
hash_uni = numpy.append(unigram_feature_vector, hash_, 1)
print(len(hash_uni))
# 28) hash bigram
hash_bi = numpy.append(bigram_feature_vector,hash_, 1)
# 29) hash tfidf
hash_tfidf = numpy.append(tfidf,hash_, 1)
# 30) hash wordvectors
hash_wordvec = numpy.append(nodoc,hash_, 1)
# 31) hash docvecs
hash_docvec = numpy.append(word_vector_feature,hash_, 1)

# 32) hash und emoji
hash_emot = numpy.append(hash_, contains_emotional_emojis, 1)

97


In [16]:
#########train and test######################

In [17]:
######logistic regression########

In [18]:
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_predict

In [19]:
X = unigram_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))


scores: [0.6        0.8        0.9        0.8        1.         0.7
 0.8        1.         1.         0.66666667]
Accuracy: 0.827 (0.138)


In [20]:
X = bigram_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.88888889 0.66666667]
Accuracy: 0.783 (0.132)


In [21]:
X = trigram_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.88888889 0.66666667]
Accuracy: 0.783 (0.132)


In [23]:
X = tfidf#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.9        0.8        0.9        1.         0.7
 0.9        0.88888889 0.88888889 0.66666667]
Accuracy: 0.824 (0.121)


In [24]:
X = post_sent_punct#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.7        0.8        0.7        0.8        0.6
 0.4        0.77777778 0.66666667 0.66666667]
Accuracy: 0.661 (0.124)


In [25]:
X = pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.6        0.9        0.7        0.8        0.7
 0.7        0.88888889 0.88888889 0.77777778]
Accuracy: 0.786 (0.102)


In [26]:
X = nodoc#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [27]:
X = word_vector_feature#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.9        0.9        0.9        1.         0.8
 0.9        1.         1.         0.55555556]
Accuracy: 0.886 (0.125)


In [28]:
##combinations##

In [29]:
#2
X = unigram_bi_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.8        0.9        0.9        1.         0.6
 0.9        1.         0.88888889 0.66666667]
Accuracy: 0.816 (0.162)


In [30]:
#3
X = uni_bi_tri_feat_vec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.8        0.8        0.9        1.         0.6
 0.9        0.88888889 0.88888889 0.66666667]
Accuracy: 0.794 (0.149)


In [31]:
#4
X = tfidf_bigram#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.88888889 0.66666667]
Accuracy: 0.783 (0.132)


In [32]:
#5
X = uni_pos_new#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.9        0.9        1.         0.7
 0.7        0.88888889 1.         0.66666667]
Accuracy: 0.816 (0.128)


In [33]:
#6
X = uni_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.7        0.9        0.8        1.         0.6
 0.7        0.77777778 1.         0.55555556]
Accuracy: 0.763 (0.154)


In [34]:
#7
X = uni_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.7        1.         1.         0.9        0.8
 0.8        0.88888889 1.         0.66666667]
Accuracy: 0.856 (0.116)


In [35]:
#8
X = bi_pos_new#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.7        0.8        0.9        0.9        0.7
 0.8        0.88888889 1.         0.77777778]
Accuracy: 0.837 (0.092)


In [26]:
#8
X = contains_emotional_emojis#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.77777778 0.77777778]
Accuracy: 0.793 (0.110)


In [36]:
#9
X = bi_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.2        0.7        0.9        0.8        1.         0.6
 0.7        0.66666667 0.88888889 0.77777778]
Accuracy: 0.723 (0.209)


In [37]:
#10
X = bi_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.7        0.9        1.         0.9        0.6
 0.8        0.88888889 1.         0.77777778]
Accuracy: 0.847 (0.121)


In [38]:
#11
X = tfidf_pos_new#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.5        0.9        0.7        0.8        0.7
 0.7        0.88888889 0.88888889 0.77777778]
Accuracy: 0.776 (0.123)


In [39]:
#12
X = tfidf_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.6        0.9        0.7        0.8        0.6
 0.5        0.77777778 0.77777778 0.77777778]
Accuracy: 0.693 (0.130)


In [41]:
#13
X = tfidf_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.6        0.9        1.         0.9        0.7
 0.7        0.88888889 0.88888889 0.77777778]
Accuracy: 0.826 (0.118)


In [42]:
#14
X = wordvec_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [43]:
#15
X = wordvec_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [44]:
#16
X = wordvec_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [45]:
#17 NICE
X = docvec_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.9        0.9        0.9        1.         0.9
 0.9        1.         1.         0.66666667]
Accuracy: 0.907 (0.092)


In [22]:
#17 NICE
X = docvec_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [1.         0.8        0.9        0.9        0.9        0.8
 0.9        1.         1.         0.55555556]
Accuracy: 0.876 (0.128)


In [47]:
#19
X = docvec_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [1.         0.8        0.9        0.9        0.9        0.9
 0.9        1.         1.         0.55555556]
Accuracy: 0.886 (0.125)


In [48]:
#20
X = pos_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.5        0.9        0.9        0.9        0.7
 0.6        0.88888889 0.88888889 0.77777778]
Accuracy: 0.796 (0.140)


In [50]:
#21
X = all_features#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [51]:
#22
X = emot_uni#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.8        0.9        0.8        1.         0.7
 0.8        1.         1.         0.66666667]
Accuracy: 0.827 (0.138)


In [52]:
#23
X = emot_bi#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.88888889 0.66666667]
Accuracy: 0.783 (0.132)


In [53]:
#24
X = emot_tfidf#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.9        0.8        0.9        1.         0.6
 0.9        0.88888889 0.77777778 0.77777778]
Accuracy: 0.824 (0.111)


In [54]:
#25
X = emot_wordvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [55]:
#26
X = emot_docvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.9        0.9        0.9        1.         0.8
 0.9        1.         1.         0.55555556]
Accuracy: 0.886 (0.125)


In [56]:
#26
X = contains_emotional_emojis#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.77777778 0.77777778]
Accuracy: 0.793 (0.110)


In [57]:
#26
X = hash_#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.8        0.9        1.         0.7
 1.         0.88888889 0.88888889 0.77777778]
Accuracy: 0.836 (0.111)


In [58]:
#26
X = hash_uni#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.8        0.9        0.8        1.         0.7
 1.         1.         1.         0.77777778]
Accuracy: 0.858 (0.137)


In [59]:
#26
X = hash_bi#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.88888889 0.66666667]
Accuracy: 0.783 (0.132)


In [60]:
#26
X = hash_tfidf#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.8        0.9        1.         0.7
 1.         1.         0.88888889 0.77777778]
Accuracy: 0.847 (0.121)


In [61]:
#26
X = hash_wordvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.9        0.9        1.         0.7
 0.8        0.55555556 0.88888889 0.66666667]
Accuracy: 0.811 (0.128)


In [62]:
#26
X = hash_docvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.9        0.9        0.9        1.         0.8
 0.9        1.         1.         0.55555556]
Accuracy: 0.886 (0.125)


In [63]:
#26
X = hash_emot#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.7        0.8        0.9        1.         0.7
 1.         0.88888889 0.77777778 0.88888889]
Accuracy: 0.846 (0.103)


In [64]:
##############naive bayes##############

In [24]:
from sklearn.naive_bayes import GaussianNB

In [66]:

X = unigram_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.6        0.6        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.154)


In [67]:

X = bigram_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.4        0.4        0.6        0.7        0.4        0.7
 0.7        0.33333333 0.66666667 0.88888889]
Accuracy: 0.579 (0.175)


In [68]:

X = trigram_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.4        0.4        0.2        0.2        0.1        0.5
 0.3        0.22222222 0.11111111 0.55555556]
Accuracy: 0.299 (0.151)


In [69]:

X = tfidf#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.6        0.5        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.166)


In [70]:
X = post_sent_punct#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.6        0.9        0.8        0.9        0.6
 0.8        0.66666667 0.88888889 0.77777778]
Accuracy: 0.743 (0.136)


In [71]:
#nice
X = pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.9        0.8        0.9        0.9        0.7
 0.9        0.88888889 1.         0.77777778]
Accuracy: 0.857 (0.081)


In [73]:

X = nodoc#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.2        0.8        0.8        1.         0.5
 0.7        0.77777778 1.         0.66666667]
Accuracy: 0.704 (0.225)


In [74]:

X = word_vector_feature#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.9        0.9        0.6        0.8        0.8
 0.7        0.88888889 0.77777778 0.66666667]
Accuracy: 0.763 (0.111)


In [75]:
##combinations##

In [76]:
#2
X = unigram_bi_feature_vector#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.5        0.7        0.7        0.8        0.9
 1.         0.55555556 0.66666667 0.77777778]
Accuracy: 0.730 (0.141)


In [77]:
#3
X = uni_bi_tri_feat_vec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.5        0.7        0.7        0.8        0.9
 1.         0.55555556 0.66666667 0.77777778]
Accuracy: 0.730 (0.141)


In [78]:
#4
X = tfidf_bigram#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.6        0.5        0.7        0.8        0.9
 1.         0.55555556 0.66666667 0.77777778]
Accuracy: 0.700 (0.161)


In [79]:
#5
X = uni_pos_new#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.6        0.6        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.154)


In [80]:
#6
X = uni_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.6        0.6        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.154)


In [81]:
#7
X = uni_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.6        0.6        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.154)


In [82]:
#8
X = bi_pos_new#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.4        0.4        0.6        0.7        0.4        0.7
 0.7        0.33333333 0.66666667 0.88888889]
Accuracy: 0.579 (0.175)


In [83]:
#9
X = bi_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.4        0.4        0.6        0.7        0.4        0.7
 0.7        0.33333333 0.66666667 0.88888889]
Accuracy: 0.579 (0.175)


In [84]:
#10
X = bi_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.4        0.4        0.6        0.7        0.4        0.7
 0.7        0.33333333 0.66666667 0.88888889]
Accuracy: 0.579 (0.175)


In [85]:
#11
X = tfidf_pos_new#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.6        0.5        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.166)


In [86]:
#12
X = tfidf_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.6        0.5        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.166)


In [87]:
#13
X = tfidf_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.6        0.5        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.166)


In [89]:
#14 
X = wordvec_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.2        0.8        0.8        1.         0.5
 0.7        0.77777778 1.         0.66666667]
Accuracy: 0.704 (0.225)


In [90]:
#15 
X = wordvec_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.2        0.8        0.8        1.         0.5
 0.7        0.77777778 1.         0.66666667]
Accuracy: 0.704 (0.225)


In [91]:
#16
X = wordvec_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()
# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.2        0.8        0.8        1.         0.5
 0.7        0.77777778 1.         0.66666667]
Accuracy: 0.704 (0.225)


In [92]:
#17
X = docvec_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.9        0.9        0.7        0.8        0.8
 0.7        0.88888889 0.88888889 0.66666667]
Accuracy: 0.784 (0.105)


In [25]:
#18
X = docvec_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.9        0.9        0.7        0.8        0.8
 0.7        0.88888889 0.77777778 0.66666667]
Accuracy: 0.773 (0.100)


In [94]:
#19
X = docvec_num_pos#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.9        0.9        0.7        0.8        0.8
 0.7        0.88888889 0.88888889 0.66666667]
Accuracy: 0.784 (0.105)


In [95]:
#20
X = pos_num#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.7        0.9        0.8        0.9        0.6
 0.8        0.88888889 1.         0.77777778]
Accuracy: 0.817 (0.108)


In [96]:
#21
X = all_features#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.5        0.6        0.8        0.9        0.9
 1.         0.44444444 0.66666667 0.77777778]
Accuracy: 0.739 (0.172)


In [97]:
#21
X = emot_uni#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.6        0.6        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.154)


In [98]:
#21
X = emot_bi#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.4        0.4        0.6        0.7        0.4        0.7
 0.7        0.33333333 0.66666667 0.88888889]
Accuracy: 0.579 (0.175)


In [99]:
#21
X = emot_tfidf#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.6        0.5        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.66666667]
Accuracy: 0.676 (0.166)


In [100]:
#21
X = emot_wordvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.2        0.8        0.8        1.         0.5
 0.7        0.77777778 0.88888889 0.66666667]
Accuracy: 0.693 (0.213)


In [101]:
#21
X = emot_docvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.9        0.9        0.7        0.8        0.8
 0.7        0.88888889 0.77777778 0.77777778]
Accuracy: 0.784 (0.093)


In [27]:
#21
X = contains_emotional_emojis#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.8        0.9        1.         0.6
 0.9        0.77777778 0.77777778 0.77777778]
Accuracy: 0.793 (0.110)


In [102]:
#21
X = hash_#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.7        0.8        0.9        1.         0.7
 1.         0.88888889 0.88888889 0.77777778]
Accuracy: 0.836 (0.111)


In [103]:
#21
X = hash_uni#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.6        0.6        0.7        0.8        0.9
 0.9        0.44444444 0.44444444 0.77777778]
Accuracy: 0.687 (0.156)


In [104]:
#21
X = hash_bi#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.4        0.6        0.7        0.4        0.7
 0.7        0.44444444 0.66666667 0.88888889]
Accuracy: 0.600 (0.152)


In [105]:
#21
X = hash_tfidf#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.6        0.5        0.7        0.8        0.9
 0.9        0.55555556 0.44444444 0.77777778]
Accuracy: 0.698 (0.156)


In [106]:
#21
X = hash_wordvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.6        0.2        0.8        0.8        1.         0.7
 0.8        0.88888889 1.         0.77777778]
Accuracy: 0.757 (0.219)


In [107]:
#21
X = hash_docvec#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.7        0.9        0.9        0.7        0.8        0.9
 0.7        0.88888889 0.88888889 0.77777778]
Accuracy: 0.816 (0.086)


In [108]:
#21
X = hash_emot#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = GaussianNB()

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.8        0.7        0.8        0.9        1.         0.7
 1.         0.88888889 0.77777778 0.88888889]
Accuracy: 0.846 (0.103)


In [19]:
#####test best features as combination########

In [34]:
#best: docvec, tfidf, hashtags
top3 = numpy.append(word_vector_feature, tfidf, 1)
top3 = numpy.append(top3, hash_, 1)


X = post_sent_punct#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.5        0.7        0.8        0.7        0.8        0.6
 0.4        0.77777778 0.66666667 0.66666667]
Accuracy: 0.661 (0.124)


In [32]:
#test leave one out 
X = post_sent_punct#data
y = handannotated_labels#target

kf = KFold(n_splits=97, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1.
 1.]
Accuracy: 0.670 (0.470)


In [37]:
#test leave one out 
X = word_vector_feature#data
y = handannotated_labels#target

kf = KFold(n_splits=10, random_state=1, shuffle=True)

# create model
model = LogisticRegression(class_weight='balanced')

# evaluate model
scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print("scores:", scores)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

scores: [0.9        0.9        0.9        0.9        1.         0.8
 0.9        1.         1.         0.55555556]
Accuracy: 0.886 (0.125)
